In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns = 99
train = pd.read_csv('/kaggle/input/DontGetKicked/training.csv')
train

In [ ]:
pd.options.display.max_columns = 99
test = pd.read_csv('/kaggle/input/DontGetKicked/test.csv')
test

In [ ]:
all_data = pd.concat([train, test], axis=0)
all_data

In [ ]:
all_data['PurchDate'] = pd.to_datetime(all_data['PurchDate'])
all_data['Year'] = all_data['PurchDate'].dt.year
all_data['Month'] = all_data['PurchDate'].dt.month
all_data['Day'] = all_data['PurchDate'].dt.day

In [ ]:
all_data_2 = all_data.drop(columns=['RefId','IsBadBuy','PurchDate'])

In [ ]:
all_data_2['Auction'].unique()

In [ ]:
#all_data_2['Auction'] = all_data_2['Auction'].replace({'ADESA':0, 'OTHER':1, 'MANHEIM':2})
#all_data_2

In [ ]:
all_data_2['Make'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_data_2['Make'] = le.fit_transform(all_data_2['Make'])
all_data_2['Make']

In [ ]:
le.classes_

In [ ]:
len(le.classes_)

In [ ]:
dict(zip(le.classes_, range(33)))

In [ ]:
category = all_data_2.columns[all_data_2.dtypes==object]
category

In [ ]:
for i in category:
    all_data_2[i] = le.fit_transform(list(all_data_2[i]))

In [ ]:
all_data_2 = all_data_2.fillna(-1)
all_data_2

In [ ]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# rfc = RandomForestClassifier(n_jobs=-1)
# rfc.fit(train_2, train['IsBadBuy'])

In [ ]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(verbose=100)
cbc.fit(train_2, train['IsBadBuy'])

In [ ]:
result = cbc.predict_proba(test_2)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/DontGetKicked/example_entry.csv')
sub

In [ ]:
sub['IsBadBuy'] = result[:, 1]
sub

In [ ]:
sub.to_csv('sub.csv', index=0)